# Image Classification with larger CNN

The goal of this exercise is to program a slightly larger CNN that incorporates some of the ideas that have been presented in the class. We use CIFAR-100, which is a bit more challenging than the MNIST data set, but not so much. It contains 50000 images that are labelled into 100 fine-grained classes of 20 more coarse grained classes. In order to make the task a mit more complicated we will use the fine grained classes.

The dataset is build into keras.

In [ ]:
import keras
import keras.datasets
import keras.datasets.cifar100
import keras.utils
import keras.optimizers
import numpy as np

import matplotlib.pyplot as plt

print(keras.__version__)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    index = np.random.randint(50000)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(np.array(x_train[index]).astype("uint8"))
    plt.title(int(y_train[index]))
    plt.axis("off")

In [ ]:
num_classes = 100
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# ResNet-style model with skip connections (Exercise 1) + Batch Normalization
def build_model():
    inp = keras.Input(shape=(32, 32, 3))

    def residual(x, filters, stride=1):
        shortcut = x
        # Conv -> BN -> ReLU
        y = keras.layers.Conv2D(filters, 3, strides=stride, padding='same', use_bias=False)(x)
        y = keras.layers.BatchNormalization()(y)
        y = keras.layers.Activation('relu')(y)
        # Conv -> BN
        y = keras.layers.Conv2D(filters, 3, padding='same', use_bias=False)(y)
        y = keras.layers.BatchNormalization()(y)
        # projection on shortcut when shape changes
        if stride != 1 or shortcut.shape[-1] != filters:
            shortcut = keras.layers.Conv2D(filters, 1, strides=stride, padding='same', use_bias=False)(shortcut)
            shortcut = keras.layers.BatchNormalization()(shortcut)
        out = keras.layers.Add()([y, shortcut])
        out = keras.layers.Activation('relu')(out)
        return out

    # Stem: Conv -> BN -> ReLU
    x = keras.layers.Conv2D(32, 3, padding='same', use_bias=False)(inp)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)

    # stage 1
    x = residual(x, 32)
    x = residual(x, 32)
    # stage 2 (downsample)
    x = residual(x, 64, stride=2)
    x = residual(x, 64)
    # stage 3 (downsample)
    x = residual(x, 128, stride=2)
    x = residual(x, 128)

    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(100, activation='softmax')(x)

    return keras.Model(inp, x)

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
batch_size = 128
epochs = 5
opt = keras.optimizers.RMSprop(learning_rate=0.01)
model = build_model()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

The performance is not bad, but also not so great. The best results without using additional data for convolutional neural networks is at about 70 percent.

In [ ]:
print(history.history)

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history['accuracy'], label='acc')
ax.plot( history.history['val_accuracy'], label='v_acc')
plt.legend()
plt.show()

## Exercise 1: Modern CNN

A *modern* CNN should have some of the features discussed in the lecture. Build a model that includes some or all of these:
- ResNet architecture with skip connections
- Batch Normalization
- L2 Regularisation
- Dropout

How does you model perform?

## Exercise 2: Monitoring

Monitoring the training is essential for long running training sessions. Include tensorboard or wandb in your training setup.

## Exercise 3: Data Augmentation (continued next lesson)

What else could be done to get better results? The dataset is quite small, so either another dataset could be used for pretraining, or data augmentation could be added....